In [1]:
pip install PyPDF2 pdf2image pytesseract Pillow python-docx spacy keras tensorflow

DEPRECATION: Loading egg at /opt/anaconda3/lib/python3.11/site-packages/fonttools-4.53.0-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 kB 3.3 MB/s eta 0:00:00
  Using cached flatbuffers-25.2.10-py2.py3-none-any.whl.metadata (875 bytes)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 1.2 MB/s eta 0:00:0000:0100:010m
   ━━━━━━━━━╸━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.4/239.5 MB 538.8 kB/s eta 0:05:39
ERROR: Exception:
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.11/site-packages/pip/_vendor/urllib3/response.py", line 438, in _error_catcher
    yield
  File "/opt/anaconda3/lib/python3.11/site-packages/pip/_vendor/urllib3/response.py", line 561, in read
    data = self._fp_read(amt) if not fp_closed else b""
           ^^^^^^^^^^^^^^^^^^
  File "/opt/an

In [3]:
pip install tensorflow

DEPRECATION: Loading egg at /opt/anaconda3/lib/python3.11/site-packages/fonttools-4.53.0-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330
  Using cached tensorflow-2.18.0-cp311-cp311-macosx_12_0_arm64.whl.metadata (4.0 kB)
  Using cached absl_py-2.1.0-py3-none-any.whl.metadata (2.3 kB)
  Using cached astunparse-1.6.3-py2.py3-none-any.whl.metadata (4.4 kB)
  Using cached flatbuffers-25.2.10-py2.py3-none-any.whl.metadata (875 bytes)
  Using cached gast-0.6.0-py3-none-any.whl.metadata (1.3 kB)
  Using cached google_pasta-0.2.0-py3-none-any.whl.metadata (814 bytes)
  Using cached libclang-18.1.1-1-py2.py3-none-macosx_11_0_arm64.whl.metadata (5.2 kB)
  Using cached opt_einsum-3.4.0-py3-none-any.whl.metadata (6.3 kB)
  Using cached termcolor-2.5.0-py3-none-any.whl.metadata (6.1 kB)
  Using cached tensorboard-2.18.0-py3-none-any.whl.metadata

In [6]:
import os
import re
import numpy as np
import pandas as pd
import PyPDF2
import docx
from PIL import Image
import pytesseract
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import spacy
import logging
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

class BatchResumeRecommendationSystem:
    def __init__(self, resume_folder, log_file='resume_recommendations.log'):
        """Initialize the recommendation system with a folder of resumes."""
        # Configure logging
        logging.basicConfig(filename=log_file, level=logging.INFO, 
                            format='%(asctime)s - %(message)s')
        
        # Load spaCy for NLP
        self.nlp = spacy.load('en_core_web_sm')
        
        # Resume folder path
        self.resume_folder = resume_folder
        
        # Expanded job listings with certifications requirements
        self.job_listings = [
            # IT Jobs
            {'title': 'Senior Software Engineer', 'company': 'TechInnovate Solutions', 'domain': 'IT',
             'skills': ['Python', 'Java', 'Machine Learning', 'AWS', 'Docker', 'DevOps'],
             'experience': '5-7 years', 'salary_range': '$120,000 - $150,000', 'location': 'San Francisco, CA',
             'certifications_required': ['aws certified', 'oracle certified']},
            {'title': 'Cloud Solutions Architect', 'company': 'CloudTech Enterprises', 'domain': 'Cloud Computing',
             'skills': ['AWS', 'Azure', 'Kubernetes', 'Terraform', 'DevOps'],
             'experience': '4-6 years', 'salary_range': '$130,000 - $160,000', 'location': 'Seattle, WA',
             'certifications_required': ['aws certified', 'azure certified', 'kubernetes certified']},
            # AI/ML Jobs
            {'title': 'AI Research Scientist', 'company': 'DeepMind Innovations', 'domain': 'AI/Machine Learning',
             'skills': ['Python', 'TensorFlow', 'PyTorch', 'Deep Learning', 'NLP'],
             'experience': '3-5 years', 'salary_range': '$140,000 - $180,000', 'location': 'San Jose, CA',
             'certifications_required': ['tensorflow developer', 'pytorch expert']},
            {'title': 'Data Scientist', 'company': 'DataDriven Co.', 'domain': 'Data Science',
             'skills': ['Python', 'R', 'SQL', 'Machine Learning', 'Statistics'],
             'experience': '2-4 years', 'salary_range': '$100,000 - $130,000', 'location': 'New York, NY',
             'certifications_required': ['certified data scientist']},
            # Environmental Jobs
            {'title': 'Environmental Data Analyst', 'company': 'GreenTech Sustainability', 'domain': 'Environmental Science',
             'skills': ['R', 'Python', 'Data Analysis', 'GIS', 'Environmental Modeling'],
             'experience': '2-4 years', 'salary_range': '$80,000 - $110,000', 'location': 'Boulder, CO',
             'certifications_required': ['leed accredited']},
            {'title': 'Sustainability Consultant', 'company': 'EcoFuture', 'domain': 'Environmental Science',
             'skills': ['Sustainability', 'Environmental Policy', 'Project Management', 'Data Analysis'],
             'experience': '3-5 years', 'salary_range': '$90,000 - $120,000', 'location': 'Portland, OR',
             'certifications_required': ['leed accredited', 'certified sustainability professional']},
            # Java Jobs
            {'title': 'Java Enterprise Architect', 'company': 'Enterprise Solutions Inc.', 'domain': 'Java Development',
             'skills': ['Java', 'Spring Boot', 'Microservices', 'RESTful APIs', 'SQL'],
             'experience': '6-8 years', 'salary_range': '$125,000 - $155,000', 'location': 'New York, NY',
             'certifications_required': ['java certified']},
            {'title': 'Java Developer', 'company': 'CodeCrafters', 'domain': 'Software Development',
             'skills': ['Java', 'Spring', 'Hibernate', 'SQL', 'Agile'],
             'experience': '3-5 years', 'salary_range': '$90,000 - $120,000', 'location': 'Austin, TX',
             'certifications_required': ['java certified']},
            # Civil Engineering Jobs
            {'title': 'Senior Civil Engineer', 'company': 'Urban Infrastructure Developers', 'domain': 'Civil Engineering',
             'skills': ['AutoCAD', 'Structural Design', 'Project Management', 'Civil 3D', 'BIM'],
             'experience': '6-8 years', 'salary_range': '$90,000 - $120,000', 'location': 'Chicago, IL',
             'certifications_required': ['pe license', 'civil engineering certification']},
            {'title': 'Structural Engineer', 'company': 'BuildStrong', 'domain': 'Civil Engineering',
             'skills': ['Structural Analysis', 'AutoCAD', 'Revit', 'Project Management'],
             'experience': '4-6 years', 'salary_range': '$85,000 - $115,000', 'location': 'Los Angeles, CA',
             'certifications_required': ['pe license']},
            # Healthcare Jobs
            {'title': 'Healthcare Data Analyst', 'company': 'HealthTech Solutions', 'domain': 'Healthcare',
             'skills': ['SQL', 'Python', 'Data Visualization', 'Healthcare Analytics'],
             'experience': '2-4 years', 'salary_range': '$80,000 - $110,000', 'location': 'Boston, MA',
             'certifications_required': ['healthcare analytics certified']},
            # Finance Jobs
            {'title': 'Financial Analyst', 'company': 'FinanceCorp', 'domain': 'Finance',
             'skills': ['Excel', 'Financial Modeling', 'Data Analysis', 'SQL'],
             'experience': '1-3 years', 'salary_range': '$70,000 - $90,000', 'location': 'Chicago, IL',
             'certifications_required': ['cfa', 'cpa']}
        ]

        self.domain_keywords = {
            'IT': [
                'software', 'technology', 'programming', 'coding', 'it', 'software development',
                'cybersecurity', 'networking', 'system administration', 'development', 'devops',
                'database management', 'tech support', 'information systems', 'IT support',
                'IT infrastructure', 'information security', 'system integration', 'scripting',
                'automation', 'virtualization', 'technical troubleshooting', 'data center', 'data management'
            ],
            'Cloud Computing': [
                'cloud', 'aws', 'azure', 'google cloud', 'infrastructure as a service', 'iaas',
                'platform as a service', 'paas', 'serverless', 'multi-cloud', 'hybrid cloud',
                'cloud migration', 'cloud security', 'cloud architecture', 'cloud orchestration',
                'cloud native', 'cloud management', 'containerization', 'kubernetes', 'docker', 'virtualization'
            ],
            'AI/Machine Learning': [
                'machine learning', 'ai', 'data science', 'deep learning', 'python', 'llms',
                'fine tuning', 'artificial intelligence', 'predictive analytics', 'big data',
                'neural networks', 'nlp', 'computer vision', 'reinforcement learning',
                'algorithm development', 'automation', 'scikit-learn', 'tensorflow', 'pytorch',
                'keras', 'supervised learning', 'unsupervised learning', 'model optimization'
            ],
            'Data Science': [
                'data science', 'data mining', 'analytics', 'r programming', 'statistical modeling',
                'data visualization', 'data wrangling', 'predictive modeling', 'big data', 'python',
                'sql', 'data engineering', 'regression analysis', 'data analysis', 'quantitative analysis',
                'data cleaning', 'business intelligence', 'dashboards', 'tableau', 'power bi',
                'data storytelling'
            ],
            'Environmental Science': [
                'environment', 'sustainability', 'climate', 'ecology', 'conservation', 'green technology',
                'waste management', 'water resources', 'environmental impact', 'environmental policy',
                'environmental compliance', 'renewable energy', 'carbon footprint', 'environmental engineering',
                'ecosystem management', 'natural resource management', 'environmental planning',
                'hazardous waste', 'climate change', 'sustainable development', 'gis', 'remote sensing'
            ],
            'Civil Engineering': [
                'civil engineering', 'construction', 'infrastructure', 'structural', 'design',
                'transportation engineering', 'geotechnical', 'construction management', 'urban planning',
                'project management', 'surveying', 'bridge design', 'road design', 'hydraulics', 'building codes',
                'cad', 'bim', 'civil drafting', 'public works', 'municipal planning', 'traffic engineering',
                'site planning', 'construction planning', 'structural analysis', 'contract management'
            ],
            'Healthcare': [
                'healthcare', 'medical', 'patient', 'clinical', 'health', 'hospital', 'patient care',
                'health informatics', 'nursing', 'diagnostics', 'treatment', 'medical research', 'pharmacy',
                'biomedical', 'emr', 'electronic health records', 'public health', 'health services',
                'healthcare administration', 'health management', 'surgery', 'emergency care',
                'healthcare compliance', 'health information technology', 'health policy', 'care management'
            ],
            'Finance': [
                'finance', 'financial', 'accounting', 'banking', 'investment', 'economics', 'financial modeling',
                'risk management', 'budgeting', 'forecasting', 'auditing', 'capital markets', 'asset management',
                'tax', 'credit analysis', 'quantitative finance', 'financial analysis', 'compliance',
                'investment banking', 'wealth management', 'equity research', 'valuation', 'corporate finance',
                'hedging', 'securities', 'derivatives', 'portfolio management', 'fiscal', 'strategic planning'
            ],
            'Marketing': [
                'marketing', 'digital marketing', 'seo', 'content creation', 'social media', 'branding', 'advertising',
                'market research', 'email marketing', 'inbound marketing', 'outbound marketing', 'analytics',
                'public relations', 'search engine marketing', 'sem', 'ppc', 'lead generation', 'content marketing',
                'copywriting', 'influencer marketing', 'brand management', 'marketing strategy', 'marketing automation',
                'customer engagement', 'growth hacking', 'conversion rate optimization', 'online advertising',
                'marketing communications'
            ],
            'Human Resources': [
                'human resources', 'hr', 'recruitment', 'employee relations', 'talent management', 'organizational development',
                'benefits administration', 'compensation', 'employee engagement', 'performance management',
                'training and development', 'labor relations', 'hr compliance', 'workforce planning', 'hris',
                'talent acquisition', 'onboarding', 'succession planning', 'hr analytics', 'employee retention',
                'diversity and inclusion', 'staffing', 'recruitment strategy', 'conflict resolution', 'employment law'
            ],
            'ATS Keywords': [
                'communication', 'teamwork', 'leadership', 'project management', 'strategic planning', 'problem solving',
                'critical thinking', 'collaboration', 'innovation', 'results-driven', 'detail-oriented', 'self-motivated',
                'adaptability', 'time management', 'multitasking', 'analytical skills', 'customer service', 'interpersonal skills',
                'decision making', 'organizational skills', 'initiative', 'accountability'
            ]
        }

        # Initialize neural network model (will be built after feature vector size is determined)
        self.input_shape = None  
        self.model = self.build_model()

    def extract_text_from_resume(self, file_path):
        """Extract text from resume files (PDF, DOCX, TXT, PNG, JPG)."""
        try:
            file_extension = os.path.splitext(file_path)[1].lower()
            if file_extension == '.pdf':
                with open(file_path, 'rb') as file:
                    reader = PyPDF2.PdfReader(file)
                    text = ' '.join([page.extract_text() for page in reader.pages if page.extract_text()])
            elif file_extension == '.docx':
                doc = docx.Document(file_path)
                text = ' '.join([paragraph.text for paragraph in doc.paragraphs])
            elif file_extension == '.txt':
                with open(file_path, 'r', encoding='utf-8') as file:
                    text = file.read()
            elif file_extension in ['.png', '.jpg', '.jpeg']:
                image = Image.open(file_path)
                text = pytesseract.image_to_string(image)
            else:
                logging.warning(f"Unsupported file type: {file_extension} for {file_path}")
                return None
            return text
        except Exception as e:
            logging.error(f"Error extracting text from {file_path}: {str(e)}")
            return None

    def extract_resume_features(self, resume_text):
        """Extract features from resume text including skills, education, experience, domains, projects, and certifications."""
        resume_text = resume_text.lower()
        doc = self.nlp(resume_text)
        
        features = {
            'skills': [],
            'education_level': None,
            'experience_years': 0,
            'domains': [],
            'project_keywords': [],
            'certifications': []
        }
        
        # Expanded list of skill patterns with additional skills
        skill_patterns = [
            'python', 'java', 'c++', 'machine learning', 'data analysis', 'aws', 'azure', 'sql', 'project management',
            'tensorflow', 'pytorch', 'cloud computing', 'spring', 'hibernate', 'autocad', 'revit', 'gis', 'r',
            'statistics', 'excel', 'financial modeling', 'docker', 'devops', 'microservices', 'kubernetes', 'terraform',
            'cybersecurity', 'network administration', 'sagemaker',
            'javascript', 'html', 'css', 'react', 'node.js', 'flask', 'django', 'php', 'sql server', 'mongodb',
            'c#', 'ruby', 'spark', 'hadoop', 'kotlin', 'swift', 'go', 'laravel', 'express.js'
        ]
        features['skills'] = [skill for skill in skill_patterns if skill in resume_text]
        
        # Education extraction based on keywords
        education_keywords = {
            'PhD': ['phd', 'doctorate'],
            'Masters': ['master', 'ms', 'ma'],
            'Bachelors': ['bachelor', 'bs', 'ba']
        }
        for level, keywords in education_keywords.items():
            if any(keyword in resume_text for keyword in keywords):
                features['education_level'] = level
                break
        
        # Extract years of experience
        experience_match = re.search(r'(\d+)\s*(?:years?|yrs?)\s*(?:of)?\s*experience', resume_text, re.IGNORECASE)
        if experience_match:
            features['experience_years'] = int(experience_match.group(1))
        
        # Domain extraction using expanded keywords
        for domain, keywords in self.domain_keywords.items():
            if any(keyword in resume_text for keyword in keywords):
                features['domains'].append(domain)
        
        # Extract project keywords
        project_keywords = re.findall(r'project\s*:\s*([^\n]+)', resume_text, re.IGNORECASE)
        features['project_keywords'] = project_keywords
        
        # Extract certifications based on common certification keywords
        certification_keywords = [
            'aws certified', 'cissp', 'oracle certified', 'microsoft certified', 'comptia', 'cisco certified',
            'tensorflow developer', 'pytorch expert', 'certified data scientist', 'leed accredited',
            'certified sustainability professional', 'java certified', 'pe license', 'civil engineering certification',
            'healthcare analytics certified', 'cfa', 'cpa', 'scrum master','snow pro certification',"CompTIA A+",
            "CompTIA Network+",
            "CompTIA Security+",
            "CompTIA Server+",
            "CompTIA Cloud+",
            "Cisco CCNA",
            "Cisco CCNP",
            "Cisco CCIE",
            "Microsoft MCSA",
            "Microsoft MCSE",
            "Microsoft Certified: Azure Fundamentals/Administrator",
            "CISSP",
            "CISM",
            "Certified Ethical Hacker (CEH)",
            "Red Hat Certified System Administrator (RHCSA)",
            "Red Hat Certified Engineer (RHCE)","IBM Data Science Professional Certificate",
            "DASCA Associate/Senior Data Scientist",
            "Coursera Data Science Specialization (Johns Hopkins University)",
            "SAS Certified Data Scientist",
            "Microsoft Certified: Azure Data Scientist Associate","Google Professional Machine Learning Engineer",
            "AWS Certified Machine Learning – Specialty",
            "Microsoft Certified: Azure AI Engineer Associate",
            "TensorFlow Developer Certificate","DataBricks","Certified Nursing Assistant (CNA)",
            "Certified Medical Assistant (CMA)",
            "Registered Health Information Technician/Administrator (RHIT/RHIA)",
            "Certified Professional in Healthcare Quality (CPHQ)",
            "Certified Health Data Analyst (CHDA)",
            "Basic Life Support (BLS)",
            "Advanced Cardiovascular Life Support (ACLS)",
            "Certified Professional in Healthcare Information and Management Systems (CPHIMS)","Chartered Financial Analyst (CFA)",
            "Certified Public Accountant (CPA)",
            "Certified Financial Planner (CFP)",
            "Financial Risk Manager (FRM)",
            "Chartered Alternative Investment Analyst (CAIA)",
            "Certified Management Accountant (CMA)",
            "Certified Treasury Professional (CTP)","Certified Paralegal (CP)",
            "Certified Legal Manager (CLM)",
            "Notary Public Certification",
            "Legal Project Management Professional (LPMP)",
            "Paralegal Advanced Competency Exam (PACE)","AWS Solutions Architect (Associate/Professional)",
            "AWS Developer",
            "AWS SysOps Administrator",
            "AWS DevOps Engineer",
            "Microsoft Azure Fundamentals",
            "Microsoft Azure Administrator",
            "Microsoft Azure Developer",
            "Microsoft Azure Solutions Architect",
            "Google Cloud Professional Cloud Architect",
            "Google Cloud Associate Cloud Engineer",
            "IBM Cloud Architect",
            "Oracle Cloud Infrastructure Certifications","Google Cloud Professional Data Engineer",
            "Microsoft Certified: Azure Data Engineer Associate",
            "Cloudera Certified Professional (CCP) Data Engineer",
            "IBM Certified Data Engineer – Big Data",
            "SAS Certified Big Data Professional","Microsoft Certified: Data Analyst Associate (Power BI)",
            "Tableau Desktop Specialist/Certified Associate",
            "Qlik Sense Business Analyst Certification",
            "SAS Certified Specialist: Visual Business Analytics",
            "Google Data Analytics Professional Certificate","LEED Accredited Professional",
            "Certified Environmental Professional (CEP)",
            "Certified Energy Manager (CEM)",
            "Certified Hazardous Materials Manager (CHMM)",
            "ISO 14001 Environmental Management Certification","Fundamentals of Engineering (FE) – Civil",
            "Professional Engineer (PE) – Civil Engineering",
            "Certified Construction Manager (CCM)",
            "Project Management Professional (PMP) for construction projects",
            "LEED Accredited Professional","Fundamentals of Engineering (FE) – Mechanical",
            "Professional Engineer (PE) – Mechanical Engineering",
            "Certified Manufacturing Engineer (CMfgE)",
            "ASME Certifications (e.g., Certified Reliability Engineer)","FAA Airframe and Powerplant (A&P) Certification",
            "Certified Aerospace Technician",
            "Professional Engineer (PE) in Aerospace",
            "Specialized Aerospace Quality Certifications (e.g., AS9100)","Project Management Professional (PMP)",
            "Six Sigma Certifications (Green Belt, Black Belt, etc.)",
            "ITIL Certification","AWS Certified Cloud Practitioner",
            "AWS Certified Solutions Architect – Associate",
            "AWS Certified Solutions Architect – Professional",
            "AWS Certified Developer – Associate",
            "AWS Certified SysOps Administrator – Associate",
            "AWS Certified DevOps Engineer – Professional",
            "AWS Certified Security – Specialty",
            "AWS Certified Machine Learning – Specialty",
            "AWS Certified Advanced Networking – Specialty",
            "AWS Certified Database – Specialty",
            "AWS Certified Data Analytics – Specialty","Microsoft Certified: Azure Fundamentals (AZ-900)",
            "Microsoft Certified: Azure Administrator Associate (AZ-104)",
            "Microsoft Certified: Azure Developer Associate (AZ-204)",
            "Microsoft Certified: Azure Security Engineer Associate (AZ-500)",
            "Microsoft Certified: Azure Data Engineer Associate (DP-203)",
            "Microsoft Certified: Azure Data Scientist Associate (DP-100)",
            "Microsoft Certified: Azure Solutions Architect Expert (AZ-305)",
            "Microsoft Certified: Azure DevOps Engineer Expert (AZ-400)",
            "Microsoft Certified: Azure AI Engineer Associate (AI-102)","Google Cloud Digital Leader",
            "Associate Cloud Engineer",
            "Professional Cloud Architect",
            "Professional Data Engineer",
            "Professional Cloud Developer",
            "Professional Cloud Network Engineer",
            "Professional Cloud Security Engineer"
        ]
        features['certifications'] = [cert for cert in certification_keywords if cert in resume_text]
        
        return features

    def vectorize_features(self, resume_features, job_listing):
        """Convert resume and job features into a numerical vector for neural network."""
        # Vectorize skills
        all_skills = set(resume_features['skills'] + job_listing['skills'])
        resume_skills_vec = [1 if skill in resume_features['skills'] else 0 for skill in all_skills]
        job_skills_vec = [1 if skill in job_listing['skills'] else 0 for skill in all_skills]
        
        # Vectorize certifications
        all_certs = set(resume_features['certifications'] + job_listing['certifications_required'])
        resume_certs_vec = [1 if cert in resume_features['certifications'] else 0 for cert in all_certs]
        job_certs_vec = [1 if cert in job_listing['certifications_required'] else 0 for cert in all_certs]
        
        # Experience difference feature
        resume_exp = resume_features['experience_years']
        job_exp_min = int(job_listing['experience'].split('-')[0])
        exp_diff = resume_exp - job_exp_min
        
        # Domain match feature
        domain_match = 1 if any(domain in job_listing['domain'].lower() for domain in resume_features['domains']) else 0
        
        feature_vector = resume_skills_vec + job_skills_vec + resume_certs_vec + job_certs_vec + [exp_diff, domain_match]
        
        if self.input_shape is None:
            self.input_shape = len(feature_vector)
            self.model = self.build_model()
        return np.array(feature_vector)

    def build_model(self):
        """Build a simple neural network model for match prediction."""
        if self.input_shape is None:
            return None  # Model will be built after input shape is determined
        model = Sequential([
            Dense(64, activation='relu', input_shape=(self.input_shape,)),
            Dense(32, activation='relu'),
            Dense(1, activation='sigmoid')
        ])
        model.compile(optimizer='adam', loss='mse')
        return model

    def calculate_job_match_score(self, resume_features, job_listing):
        """Calculate match score using a rule-based approach with added certifications bonus."""
        feature_vector = self.vectorize_features(resume_features, job_listing)
        # Rule-based scoring:
        score = 0
        
        # Skill match: up to 30 points
        skill_match = len(set(resume_features['skills']) & set(job_listing['skills']))
        if job_listing['skills']:
            score += (skill_match / len(job_listing['skills'])) * 30
        
        # Experience match: bonus points for closeness to minimum required experience (up to 20 points)
        exp_range = job_listing['experience'].split('-')
        min_exp = int(exp_range[0])
        resume_exp = resume_features['experience_years']
        score += max(0, 20 - abs(resume_exp - min_exp))
        
        # Domain match: 20 points if any domain matches
        domain_match = any(domain.lower() in job_listing['domain'].lower() for domain in resume_features['domains'])
        score += 20 if domain_match else 0
        
        # Project experience: 20 points if project keywords are found
        score += 20 if resume_features['project_keywords'] else 0
        
        # Education bonus
        education_bonus = {'Bachelors': 5, 'Masters': 7, 'PhD': 10}
        if resume_features['education_level']:
            score += education_bonus.get(resume_features['education_level'], 0)
        
        # Certification match bonus: if job requires certifications, add bonus proportionally (up to 20 points)
        if job_listing['certifications_required']:
            matching_certs = set(resume_features['certifications']) & set(job_listing['certifications_required'])
            cert_bonus = (len(matching_certs) / len(job_listing['certifications_required'])) * 20
            score += cert_bonus
        
        # Ensure score does not exceed 100
        return min(score, 100)

    def recommend_jobs(self, resume_features):
        """Recommend jobs based on resume features."""
        job_recommendations = []
        for job in self.job_listings:
            match_score = self.calculate_job_match_score(resume_features, job)
            job_recommendations.append({
                'job_title': job['title'],
                'company': job['company'],
                'domain': job['domain'],
                'salary_range': job['salary_range'],
                'location': job['location'],
                'match_score': match_score
            })
        job_recommendations.sort(key=lambda x: x['match_score'], reverse=True)
        return job_recommendations

    def process_resume_folder(self):
        """Process all resumes and return recommendations in a DataFrame."""
        all_recommendations = []
        for filename in os.listdir(self.resume_folder):
            file_path = os.path.join(self.resume_folder, filename)
            if os.path.isdir(file_path):
                continue
            
            resume_text = self.extract_text_from_resume(file_path)
            if not resume_text:
                logging.warning(f"Could not process resume: {filename}")
                continue
            
            resume_features = self.extract_resume_features(resume_text)
            job_recommendations = self.recommend_jobs(resume_features)
            
            for rec in job_recommendations[:5]:  # Top 5 recommendations per resume
                all_recommendations.append({
                    'resume': filename,
                    'job_title': rec['job_title'],
                    'company': rec['company'],
                    'domain': rec['domain'],
                    'salary_range': rec['salary_range'],
                    'location': rec['location'],
                    'match_score': rec['match_score']
                })
            
            logging.info(f"Recommendations for {filename}:")
            for rec in job_recommendations[:3]:
                logging.info(f"  - {rec['job_title']} (Match Score: {rec['match_score']:.2f})")
        
        return pd.DataFrame(all_recommendations)

def main():
    resume_folder = '/Users/srikar/Desktop/KRIGNAL'
    recommender = BatchResumeRecommendationSystem(resume_folder)
    recommendations_df = recommender.process_resume_folder()
    print("\nRecommendations DataFrame:")
    print(recommendations_df)
    # Optionally save to CSV
    recommendations_df.to_csv('recommendations.csv', index=False)

if __name__ == "__main__":
    main()


/opt/anaconda3/lib/python3.11/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)



Recommendations DataFrame:
                                 resume                   job_title  \
0   Anudeep_MachineLearningEngineer.pdf           Financial Analyst   
1   Anudeep_MachineLearningEngineer.pdf              Data Scientist   
2   Anudeep_MachineLearningEngineer.pdf  Environmental Data Analyst   
3   Anudeep_MachineLearningEngineer.pdf       AI Research Scientist   
4   Anudeep_MachineLearningEngineer.pdf   Sustainability Consultant   
5                              java.png       AI Research Scientist   
6                              java.png         Structural Engineer   
7                              java.png    Senior Software Engineer   
8                              java.png       Senior Civil Engineer   
9                              java.png           Financial Analyst   
10                           ASR_SD.pdf           Financial Analyst   
11                           ASR_SD.pdf              Data Scientist   
12                           ASR_SD.pdf  Environm

In [ ]:
import os
import re
import time
import requests
import pandas as pd
import spacy
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import PyPDF2
import docx
from PIL import Image
import pytesseract

# Load spaCy for NLP tasks
nlp = spacy.load('en_core_web_sm')

# --- Job Scraping Functions ---

def scroll_to_load_all(driver):
    """Scroll the page to load all job listings dynamically."""
    last_height = driver.execute_script("return document.body.scrollHeight")
    while True:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(3)
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

def is_valid_link(url):
    """Validate if a URL is accessible."""
    try:
        response = requests.head(url, timeout=5)
        return response.status_code == 200
    except requests.RequestException:
        return False

def extract_job_features(job_description):
    """Extract skills, experience, and certifications from job description text."""
    doc = nlp(job_description.lower())
    features = {'skills': [], 'experience': 0, 'certifications_required': []}

    # Common skills list (expandable)
    skills_list = [
        'python', 'java', 'c++', 'machine learning', 'data analysis', 'aws', 'azure', 'sql',
        'tensorflow', 'pytorch', 'docker', 'devops', 'kubernetes', 'javascript', 'spring', 'hibernate'
    ]
    features['skills'] = [skill for skill in skills_list if skill in job_description.lower()]

    # Extract experience (e.g., "3 years of experience")
    exp_match = re.search(r'(\d+)\s*years?\s*of\s*experience', job_description, re.IGNORECASE)
    if exp_match:
        features['experience'] = int(exp_match.group(1))

    # Common certifications list (expandable)
    cert_list = ['aws certified', 'cissp', 'oracle certified', 'java certified', 'pe license']
    features['certifications_required'] = [cert for cert in cert_list if cert.lower() in job_description.lower()]

    return features

def scrape_zscaler_jobs(job_keywords):
    """Scrape job listings from Zscaler and extract detailed features."""
    options = webdriver.ChromeOptions()
    options.add_argument('--headless')
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
    jobs_data = []
    base_url = "https://boards.greenhouse.io/zscaler"
    company_name = "Zscaler"

    try:
        driver.get(base_url)
        print(f"Scraping jobs from Zscaler with keywords '{job_keywords}'...")
        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, "opening")))
        scroll_to_load_all(driver)
        job_elements = driver.find_elements(By.CLASS_NAME, "opening")

        for job in job_elements:
            try:
                title = job.find_element(By.CSS_SELECTOR, "a").text.strip()
                link = job.find_element(By.CSS_SELECTOR, "a").get_attribute("href")
                location = job.find_element(By.CLASS_NAME, "location").text.strip() if job.find_elements(By.CLASS_NAME, "location") else "Not specified"

                if job_keywords.lower() in title.lower() and is_valid_link(link):
                    # Visit job detail page to get description
                    driver.get(link)
                    try:
                        WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.TAG_NAME, "div")))
                        description_elem = driver.find_element(By.CLASS_NAME, "app-body")  # Adjust based on Zscaler page structure
                        description = description_elem.text
                    except:
                        description = "Description not available"
                    driver.back()
                    time.sleep(1)  # Prevent overwhelming the server

                    # Extract features from description
                    job_features = extract_job_features(description)

                    jobs_data.append({
                        'title': title,
                        'company': company_name,
                        'domain': 'Not specified',
                        'skills': job_features['skills'],
                        'experience': f"{job_features['experience']} years",
                        'salary_range': 'Not specified',
                        'location': location,
                        'certifications_required': job_features['certifications_required'],
                        'link': link
                    })
            except Exception as e:
                print(f"Error parsing Zscaler job: {e}")
                continue

        print(f"Found {len(jobs_data)} jobs from Zscaler.")
    except Exception as e:
        print(f"Error during Zscaler scraping: {e}")
    finally:
        driver.quit()
    return jobs_data

def scrape_amazon_jobs(job_keywords):
    """Scrape job listings from Amazon and extract detailed features."""
    options = webdriver.ChromeOptions()
    options.add_argument('--headless')
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
    jobs_data = []
    base_url = "https://www.amazon.jobs/en/search?base_query={}"
    search_url = base_url.format(job_keywords.replace(" ", "+"))
    company_name = "Amazon"

    try:
        driver.get(search_url)
        print(f"Scraping jobs from Amazon with keywords '{job_keywords}'...")
        try:
            WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.XPATH, "//button[contains(text(), 'Accept')]"))).click()
        except:
            pass
        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, "job-tile")))
        scroll_to_load_all(driver)
        job_elements = driver.find_elements(By.CLASS_NAME, "job-tile")

        for job in job_elements:
            try:
                title = job.find_element(By.CLASS_NAME, "job-title").text.strip()
                link = job.find_element(By.TAG_NAME, "a").get_attribute("href")
                location = job.find_element(By.CLASS_NAME, "location-and-id").text.strip() if job.find_elements(By.CLASS_NAME, "location-and-id") else "Not specified"

                if job_keywords.lower() in title.lower() and is_valid_link(link):
                    # Visit job detail page to get description
                    driver.get(link)
                    try:
                        WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.CLASS_NAME, "description")))
                        description_elem = driver.find_element(By.CLASS_NAME, "description")
                        description = description_elem.text
                    except:
                        description = "Description not available"
                    driver.back()
                    time.sleep(1)

                    # Extract features from description
                    job_features = extract_job_features(description)

                    jobs_data.append({
                        'title': title,
                        'company': company_name,
                        'domain': 'Not specified',
                        'skills': job_features['skills'],
                        'experience': f"{job_features['experience']} years",
                        'salary_range': 'Not specified',
                        'location': location,
                        'certifications_required': job_features['certifications_required'],
                        'link': link
                    })
            except Exception as e:
                print(f"Error parsing Amazon job: {e}")
                continue

        print(f"Found {len(jobs_data)} jobs from Amazon.")
    except Exception as e:
        print(f"Error during Amazon scraping: {e}")
    finally:
        driver.quit()
    return jobs_data

# --- Resume Processing and Recommendation Functions ---

def extract_text_from_resume(file_path):
    """Extract text from a resume file (PDF, DOCX, TXT, PNG, JPG)."""
    file_extension = os.path.splitext(file_path)[1].lower()
    try:
        if file_extension == '.pdf':
            with open(file_path, 'rb') as file:
                reader = PyPDF2.PdfReader(file)
                return ' '.join([page.extract_text() for page in reader.pages if page.extract_text()])
        elif file_extension == '.docx':
            doc = docx.Document(file_path)
            return ' '.join([paragraph.text for paragraph in doc.paragraphs])
        elif file_extension == '.txt':
            with open(file_path, 'r', encoding='utf-8') as file:
                return file.read()
        elif file_extension in ['.png', '.jpg', '.jpeg']:
            image = Image.open(file_path)
            return pytesseract.image_to_string(image)
        else:
            print(f"Unsupported file type: {file_extension}")
            return None
    except Exception as e:
        print(f"Error extracting text from resume: {e}")
        return None

def extract_resume_features(resume_text):
    """Extract features from resume text."""
    resume_text = resume_text.lower()
    doc = nlp(resume_text)
    features = {
        'skills': [],
        'experience_years': 0,
        'domains': [],
        'project_keywords': [],
        'certifications': []
    }

    # Skills extraction
    skill_patterns = [
        'python', 'java', 'c++', 'machine learning', 'data analysis', 'aws', 'azure', 'sql',
        'tensorflow', 'pytorch', 'docker', 'devops', 'kubernetes', 'javascript', 'spring'
    ]
    features['skills'] = [skill for skill in skill_patterns if skill in resume_text]

    # Experience extraction
    exp_match = re.search(r'(\d+)\s*(?:years?|yrs?)\s*(?:of)?\s*experience', resume_text, re.IGNORECASE)
    if exp_match:
        features['experience_years'] = int(exp_match.group(1))

    # Certifications extraction
    cert_keywords = ['aws certified', 'cissp', 'oracle certified', 'java certified', 'pe license']
    features['certifications'] = [cert for cert in cert_keywords if cert in resume_text]

    # Placeholder for domains and project keywords (simplified)
    features['domains'] = ['IT']  # Default domain; can be enhanced
    features['project_keywords'] = re.findall(r'project\s*:\s*([^\n]+)', resume_text, re.IGNORECASE)

    return features

def calculate_job_match_score(resume_features, job_listing):
    """Calculate a match score between resume features and job requirements."""
    score = 0

    # Skill match (up to 30 points)
    skill_match = len(set(resume_features['skills']) & set(job_listing['skills']))
    if job_listing['skills']:
        score += (skill_match / len(job_listing['skills'])) * 30

    # Experience match (up to 20 points)
    job_exp = int(job_listing['experience'].split()[0])  # Extract minimum experience
    resume_exp = resume_features['experience_years']
    score += max(0, 20 - abs(resume_exp - job_exp))

    # Certification match (up to 20 points)
    if job_listing['certifications_required']:
        cert_match = len(set(resume_features['certifications']) & set(job_listing['certifications_required']))
        score += (cert_match / len(job_listing['certifications_required'])) * 20 if job_listing['certifications_required'] else 0

    # Project bonus (20 points if projects exist)
    score += 20 if resume_features['project_keywords'] else 0

    return min(score, 100)

def recommend_jobs_for_resume(resume_path, job_listings):
    """Recommend jobs based on a single resume and a list of job listings."""
    resume_text = extract_text_from_resume(resume_path)
    if not resume_text:
        return []

    resume_features = extract_resume_features(resume_text)
    recommendations = []

    for job in job_listings:
        match_score = calculate_job_match_score(resume_features, job)
        recommendations.append({
            'job_title': job['title'],
            'company': job['company'],
            'location': job['location'],
            'link': job['link'],
            'match_score': match_score
        })

    recommendations.sort(key=lambda x: x['match_score'], reverse=True)
    return recommendations

# --- AI Agent Interface ---

def job_recommendation_agent():
    """Interactive AI agent to guide the user through job scraping and recommendation."""
    print("Welcome to the Job Recommendation Agent!")
    print("I will help you find job matches from Zscaler and Amazon based on your interests and resume.")

    # Step 1: Get job keywords
    job_keywords = input("\nPlease enter your desired job title or keywords (e.g., Software Engineer): ").strip()
    if not job_keywords:
        print("No keywords provided. Using default: 'Engineer'.")
        job_keywords = "Engineer"

    # Step 2: Scrape jobs
    print("\nScraping jobs... This may take a few minutes.")
    zscaler_jobs = scrape_zscaler_jobs(job_keywords)
    amazon_jobs = scrape_amazon_jobs(job_keywords)
    all_jobs = zscaler_jobs + amazon_jobs
    print(f"\nTotal jobs found: {len(all_jobs)}")

    if not all_jobs:
        print("No jobs found matching your keywords. Please try different keywords.")
        return

    # Step 3: Get resume
    resume_path = input("\nPlease enter the full path to your resume file (PDF, DOCX, TXT, PNG, JPG): ").strip()
    if not os.path.exists(resume_path):
        print("Resume file not found. Please check the path and try again.")
        return

    # Step 4: Generate recommendations
    print("\nProcessing your resume and generating recommendations...")
    recommendations = recommend_jobs_for_resume(resume_path, all_jobs)

    if not recommendations:
        print("Could not process your resume or no matches found.")
        return

    # Step 5: Display top recommendations
    print("\nTop 5 Job Recommendations:")
    for i, rec in enumerate(recommendations[:5], 1):
        print(f"{i}. {rec['job_title']} at {rec['company']}")
        print(f"   Location: {rec['location']}")
        print(f"   Match Score: {rec['match_score']:.2f}")
        print(f"   Link: {rec['link']}")
        print("-" * 50)

    # Optional: Save to CSV
    df = pd.DataFrame(recommendations)
    filename = f"job_recommendations_{job_keywords.replace(' ', '_')}.csv"
    df.to_csv(filename, index=False)
    print(f"\nAll recommendations saved to '{filename}'")

# --- Run the Agent ---

if __name__ == "__main__":
    job_recommendation_agent()

Welcome to the Job Recommendation Agent!
I will help you find job matches from Zscaler and Amazon based on your interests and resume.



Please enter your desired job title or keywords (e.g., Software Engineer):  Software Engineer



Scraping jobs... This may take a few minutes.
Scraping jobs from Zscaler with keywords 'Software Engineer'...
